In [3]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

def geocode_address(address):
    """
    Geocode an address using Nominatim and return the latitude and longitude.
    """
    geolocator = Nominatim(user_agent="APIEx")
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return "No location found"
    except GeocoderTimedOut:
        return "Geocoder Timed Out"

# Address to be geocoded; address for Mission Hospitals Ashville, NC, only trauma ctr in Mounain Area RAC
address = "509 Biltmore Ave, Asheville, NC 28801, United States"

# Geocoding the address
coords = geocode_address(address)

print(f"Coordinates for '{address}': {coords}")


Coordinates for '509 Biltmore Ave, Asheville, NC 28801, United States': (35.5766889, -82.54922485627307)


Geocoded coords for one example trauma center (Mission Hospitals, Asheville, NC)

Calculate Geographical Centroids for Mountain Area RAC:

In [10]:
import geopandas as gpd

# Path to the shapefile
shapefile_path = r"C:\Users\Danie\Downloads\North_Carolina_State_and_County_Boundary_Polygons\North_Carolina_State_and_County_Boundary_Polygons.shp"


# Read the shapefile
counties = gpd.read_file(shapefile_path)
counties = counties.to_crs(epsg=4326)
# List of counties in the "Mountain Area Trauma Regional Advisory Committee"
mountain_counties = [
    'Madison', 'Yancey', 'Mitchell', 'Avery',
    'Buncombe', 'Haywood', 'Henderson', 'Swain',
    'Jackson', 'Macon', 'Graham', 'Cherokee',
    'Clay', 'Transylvania', 'McDowell', 'Rutherford',
    'Polk', 'Burke'
]

# Filter for the counties of interest
mountain_counties_gdf = counties[counties['County'].isin(mountain_counties)]

# Calculate the centroids
mountain_counties_gdf['centroid'] = mountain_counties_gdf.geometry.centroid

# Extract the centroid coordinates
mountain_counties_gdf['centroid_x'] = mountain_counties_gdf.centroid.x #longitude
mountain_counties_gdf['centroid_y'] = mountain_counties_gdf.centroid.y #latitude
# Print out the results
for index, row in mountain_counties_gdf.iterrows():
    print(f"{row['County']} County centroid: ({row['centroid_y']}, {row['centroid_x']})") #printed so latitude first, longitude second


Jackson County centroid: (35.2875876850383, -83.14085006975861)
Transylvania County centroid: (35.20217768788943, -82.79830045948819)
Macon County centroid: (35.150468365516694, -83.42217953577138)
Cherokee County centroid: (35.133857008717825, -84.06344345926534)
Clay County centroid: (35.057180321122594, -83.75017461869042)
Henderson County centroid: (35.336255896657065, -82.4799620000253)
Rutherford County centroid: (35.40259428326886, -81.91983104221609)
Polk County centroid: (35.27928036969197, -82.16968036211972)
Graham County centroid: (35.350172319516766, -83.8335821689321)
Haywood County centroid: (35.55605574881402, -82.98217308131721)
Swain County centroid: (35.48681570474419, -83.49273867808799)
Buncombe County centroid: (35.611209098793495, -82.53010657473348)
McDowell County centroid: (35.68163641105261, -82.04931697443561)
Burke County centroid: (35.749644249397626, -81.70484999912061)
Madison County centroid: (35.85803560367825, -82.7057582490002)
Yancey County centroid

C:\Users\Danie\AppData\Local\Temp\ipykernel_1396\672052659.py:23: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mountain_counties_gdf['centroid'] = mountain_counties_gdf.geometry.centroid
C:\Users\Danie\miniconda3\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\Danie\AppData\Local\Temp\ipykernel_1396\672052659.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mountain_counties_gd